In [1]:
import sys
sys.argv = [sys.argv[0]]  

In [2]:
import csv
import argparse
import os
import time
import socket
import ssl
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import re
import subprocess

In [ ]:
def measure_rtt(url):
    try:
        ip_address = socket.gethostbyname(url)

        tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        tcp_start_time = time.time()
        tcp_socket.connect((ip_address, 80))
        tcp_end_time = time.time()
        tcp_rtt = (tcp_end_time - tcp_start_time) * 1000
        tcp_socket.close()

        rtt = tcp_rtt

        return rtt
    except Exception as e:
        print(f"Error measuring RTT: {e}")
        return None



def parse_arguments():
    parser = argparse.ArgumentParser(
        description="Get variable from python script",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )

    parser.add_argument("--geckodriver", "-g", default="/home/........................../geckodriver", help="geckodriver executable")
    parser.add_argument("--extension-xpi", "-e", default="/home/......................./extension/wasm.xpi", help="compressed extension (.xpi file)")
    parser.add_argument("--output-dir", "-o", default="/home/........................./output/", help="output directory to store extension events")

    return parser.parse_args()


def connection_establish(url):
    try:
        ip_address = socket.gethostbyname(url)

        tls_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        tls_context = ssl.create_default_context()
        tls_start_time = time.time()
        tls_socket = tls_context.wrap_socket(tls_socket, server_hostname=url)
        tls_socket.connect((ip_address, 443))
        tls_end_time = time.time()
        tls_rtt = (tls_end_time - tls_start_time) * 1000
        tls_socket.close()


        rtt_http = tls_rtt

        return rtt_http
    except Exception as e:
        print(f"Error measuring RTT: {e}")
        return None


def measure_website_performance(driver, website):
    
    try:
        
        response = requests.get('https://' + website)
        rtt_http = connection_establish(website)
        rtt_tcp_tls = measure_rtt(website)
        
        driver.get(response.url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
        initial_response = driver.execute_script("return performance.timing.responseStart - performance.timing.requestStart")
        latency = rtt_tcp_tls/2
        page_load_time = driver.execute_script("return performance.timing.loadEventEnd - performance.timing.navigationStart")#no extension/wasm/fpki
        dependency_load_time = driver.execute_script("return performance.timing.domContentLoadedEventEnd - performance.timing.navigationStart")
        page_size = len(driver.page_source.encode('utf-8'))
        return rtt_http, rtt_tcp_tls, initial_response, latency, page_load_time, dependency_load_time, page_size
    except Exception as e:
        print(f"Error measuring website performance for {website}: {e}")
        return None, None, None, None, None, None, None


def main():
    args = parse_arguments()

    countries = ["Albania", "Argentina", "Australia", "Austria", "Belgium", "Bosnia_And_Herzegovina","Brazil","Bulgaria","Canada","Chile","Colombia","Costa_Rica","Croatia","Cyprus","Czech_Republic","Denmark","Estonia","Finland","France","Germany","Greece","Hong_Kong","Hungary","Iceland","Indonesia","Ireland","Israel","Italy","Japan","Latvia","Lithuania","Luxembourg","Malaysia","Mexico","Moldova","Netherlands","New_Zealand","North_Macedonia","Poland","Portugal","Romania","Serbia","Singapore","Slovakia","Slovenia","South_Africa","South_Korea","Spain","Sweden","Switzerland","Taiwan","Thailand","Turkey","Ukraine","United_Kingdom","United_States","Vietnam","Georgia","Norway"]

    multiple_times = 10  

    for country_choice in countries:
        os.system(f"nordvpn connect {country_choice}")
        try:
            with open("websites_names.csv") as f:
                reader = csv.reader(f)
                next(reader) 
                for row in reader:
                    website = row[0]

                    executable_path = args.geckodriver
                    path = os.path.abspath(args.extension_xpi)

                    for repetition in range(multiple_times):
                        driver = None 
                        try:
                            driver = webdriver.Firefox(executable_path=executable_path)
                            driver.install_addon(path, temporary=True)

                            csv_file = os.path.join(args.output_dir, "metrics.csv")

                            if not os.path.exists(csv_file):
                                with open(csv_file, mode='w') as metrics_file:
                                    metrics_writer = csv.writer(metrics_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                    metrics_writer.writerow(['Website', 'Country', 'Connection Establishment Time', 'RTT', 'Initial Response', 'Latency', 'Page Load Time', 'Dependency Load Time', 'Page Size', 'IP', 'City', 'Country', 'Organization'])

                            print(f"Calculating performance for {website} in {country_choice}... Repetition {repetition + 1}/{multiple_times}")
                            rtt_http, rtt_tcp_tls, initial_response, latency, page_load_time, dependency_load_time, page_size = measure_website_performance(driver, website)

                            response = requests.get("https://ipinfo.io/json")
                            ip = response.json()['ip']
                            city = response.json()['city']
                            country = response.json()['country']
                            organization = response.json()['org']

                            print(f"{website}: Connection Establishment Time={rtt_http}, RTT={rtt_tcp_tls}, Initial Response={initial_response}, Latency={latency}, Page Load Time={page_load_time}, Dependency Load Time={dependency_load_time}, Page Size={page_size}, IP={ip}, City={city}, Country={country}, Organization={organization}")

                            with open(csv_file, mode='a') as metrics_file:
                                metrics_writer = csv.writer(metrics_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                                metrics_writer.writerow([website, country_choice, rtt_http, rtt_tcp_tls, initial_response, latency, page_load_time, dependency_load_time, page_size, ip, city, country, organization])

                            driver.execute_script("""
                                window.extensionlogs = [];
                                document.addEventListener(
                                    "pageloadfinished",
                                    function(e){
                                        const ep = JSON.parse(e.detail);
                                        window.extensionContextLogEntries = ep;
                                    }, false);
                            """)

                            wait_time = 0
                            while wait_time < 10:
                                log = driver.execute_script("""
                                    const tmp = window.extensionContextLogEntries;
                                    window.extensionContextLogEntries = null;
                                    return tmp;
                                """)
                                if log is not None:
                                    print(f"Got extension event for {website} in {country_choice}: {log}")
                                    output_file = os.path.join(args.output_dir, f"{website}_{country_choice}_{repetition + 1}.json")
                                    with open(output_file, 'w') as f:
                                        json.dump(log, f)
                                    break
                                print("Waiting for new log...")
                                time.sleep(0.3)
                                wait_time += 0.3

                        except Exception as e:
                            print(f"Error occurred: {e}")
                        finally:
                            if driver:
                                driver.quit() 

        except Exception as e:
            print(f"Error reading websites CSV: {e}")
        finally:
            os.system("nordvpn disconnect")

if __name__ == '__main__':
    main()
